<a href="https://colab.research.google.com/github/alaaHh27/nlpTask/blob/main/Copy_of_nlpTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

# Set hyperparameters
vocab_size = 10000   # Keep the top 10,000 most frequent words
max_length = 200     # Maximum review length
embedding_dim = 32
batch_size = 128
epochs = 5

# Load IMDb dataset
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

# Pad sequences to make them uniform
train_data = pad_sequences(train_data, maxlen=max_length, padding='post')
test_data = pad_sequences(test_data, maxlen=max_length, padding='post')

# Build the model
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Conv1D(64, 5, activation='relu'),
    keras.layers.MaxPooling1D(pool_size=2),
    keras.layers.LSTM(64, return_sequences=True),
    keras.layers.LSTM(32),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
print("Training the model...")
model.fit(train_data, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(test_data, test_labels))

# Save the model
model.save("sentiment_model.h5")
print("Model saved as sentiment_model.h5")

# Load IMDb word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for key, value in word_index.items()}

# Function to convert text to sequence
def text_to_sequence(text):
    words = text.lower().split()
    sequence = [word_index.get(word, 2) for word in words]  # 2 is the index for unknown words
    return pad_sequences([sequence], maxlen=max_length, padding='post')

# Function to predict sentiment
def predict_sentiment(review):
    sequence = text_to_sequence(review)
    prediction = model.predict(sequence)[0][0]
    sentiment = "Positive" if prediction > 0.5 else "Negative"
    return f"Predicted Sentiment: {sentiment} (Confidence: {prediction:.4f})"

# Test the model with a user review
user_review = input("Enter a movie review: ")
print(predict_sentiment(user_review))